## Robustness of European transportation

In [ ]:
import networkx as nx

# Load the Pajek graph and convert to a simple undirected NetworkX graph
G0 = nx.read_pajek("data/europe.net")
G  = nx.Graph(G0)

n, m = G.number_of_nodes(), G.number_of_edges()
print(f"Nodes {n} and edges {m}")


Nodes 1039 and edges 1305


Helper fucntion. We measure how big the largest island of connected cities is and if the graph is empty the size is zero.

In [3]:
def lcc_size(H: nx.Graph) -> int:
    if H.number_of_nodes() == 0:
        return 0
    return len(max(nx.connected_components(H), key=len))


Baseline connectivity. Just to know where we start.

In [4]:
baseline_lcc = lcc_size(G)
print(f"LCC before removals {baseline_lcc} which is {baseline_lcc/n:.1%} of all cities")

LCC before removals 1039 which is 100.0% of all cities


The strategy I will use is a greedy corridor strategy based on betweenness centrality. Idea  is that cities that lie on many shortest routes act like corridors between large regions. At each step I will compute betweenness on the current graph remove the city with the highest score then repeat. This is fast and it naturally targets bridge like cities that keep distant regions connected.

In [5]:
def greedy_betweenness_k_removal(G: nx.Graph, k: int = 15):
    H = G.copy()
    removed = []
    for _ in range(k):
        # recompute on the current graph at every step
        bc = nx.betweenness_centrality(H)
        v  = max(bc, key=bc.get)
        removed.append(v)
        H.remove_node(v)
    return removed, H

removed, G_after = greedy_betweenness_k_removal(G, k=15)
removed


['Brest',
 'Warsaw',
 'Saint Petersburg',
 'Kiev',
 'Niš',
 'Gdańsk',
 'Mukachevo',
 'Chişinău',
 'Trieste',
 'Vinnytsia',
 'Budapest',
 'Zagreb',
 'Oradea',
 'Kherson',
 'Suceava']

Lets now check the size of the largest island after the removals of the above cities and see the share among remaining cities and among the original graph for completeness.

In [ ]:
lcc_after = lcc_size(G_after)
remaining = G_after.number_of_nodes()           # =n-15
share_remaining = lcc_after / remaining
share_original  = lcc_after / n

print("Removed cities in order:")
print(", ".join(removed))
print(f"LCC after removals {lcc_after}")
print(f"Share of remaining cities {share_remaining:.1%}")



Removed cities in order:
Brest, Warsaw, Saint Petersburg, Kiev, Niš, Gdańsk, Mukachevo, Chişinău, Trieste, Vinnytsia, Budapest, Zagreb, Oradea, Kherson, Suceava
LCC after removals 442
Share of remaining cities 43.2%


Therefore cities are 
1. Brest
2. Warsaw
3. Saint Petersburg 
4. Kiev, 
5. Niš 
6. Gdańsk 
7. Mukachevo
8. Chişinău 
9.Trieste
10. Vinnytsia
11. Budapest, 
12. Zagreb
13. Oradea 
14. Kherson 
15. Suceava

Largest connected component after removals is 442 cities which is 43.2 percent of the remaining cities



Explaintion / Resasoning:

Some cities serve as corridors that many shortest trips pass through.If I close those corridor cities the map falls into separate islands and travel across Europe becomes hard. Betweenness centrality scores how often a city sits on shortest routes. I repeatedly remove the highest betweenness city and recompute because the network changes after every removal. After removing fifteen such cities the largest remaining island contains well under 50% of the remaining cities so the goal is met.